# 2. 02_silver_transformations — Camada Silver (Clean / Curated)

## 2.1 Objetivo deste notebook
Este notebook transforma a tabela **Bronze** em uma tabela **Silver**, com:
- tipagem e padronização de atributos;
- criação de campos derivados;
- extração de atributos do PGN (ex.: Date/UTCDate, ECO, Termination, Link);
- validações de qualidade (domínios, nulos, duplicidade).

## 2.2 Regras principais
- `rating_diff = white_rating - black_rating`
- `outcome`:
  - `white_win` se `white_result = "win"`
  - `black_win` se `black_result = "win"`
  - `draw` se ambos os lados indicarem motivo de empate (ex.: stalemate, repetition, agreed, insufficient, 50move, timevsinsufficient)
  - `unknown` caso contrário (para evidenciar inconsistências de domínio)
- `score`: vitória = 1, empate = 0.5, derrota = 0

## 2.3 Evidências
Este notebook apresenta:
- contagem antes/depois de deduplicação;
- schema e amostra da Silver;
- checagens de nulos, domínios e duplicidade (por `game_key`).


## 2.4 Importar Bronze e definir datatypes

In [0]:
from pyspark.sql import functions as F

# Bronze Table
bronze_table = "workspace.default.bronze_games"
df_bronze = spark.table(bronze_table)
print("Bronze table:", bronze_table)

# Definir Datatypes
type_map = {
    "white_rating": "int",
    "black_rating": "int",
    "rated": "boolean",
    "white_id": "string",
    "black_id": "string",
    "time_class": "string",
    "time_control": "string",
    "rules": "string",
    "white_result": "string",
    "black_result": "string",
    "pgn": "string"
}

df_silver = df_bronze
for col_name, spark_type in type_map.items():
    if col_name in df_silver.columns:
        df_silver = df_silver.withColumn(col_name, F.col(col_name).cast(spark_type))

display(df_silver.limit(10))
df_silver.printSchema()


Bronze table: workspace.default.bronze_games


white_username,black_username,white_id,black_id,white_rating,black_rating,white_result,black_result,time_class,time_control,rules,rated,fen,pgn
-Amos-,miniman2804,https://api.chess.com/pub/player/-amos-,https://api.chess.com/pub/player/miniman2804,1708,1608,win,checkmated,daily,1/259200,chess,true,r2r4/p2p1p1p/b6R/n1p1kp2/2P2P2/3BP3/PP5P/4K2R b K f3 1 22,"[Event ""Enjoyable games 2 - Round 1""] [Site ""Chess.com""] [Date ""2013.01.30""] [Round ""-""] [White ""-Amos-""] [Black ""miniman2804""] [Result ""1-0""] [Tournament ""https://www.chess.com/tournament/enjoyable-games-2""] [CurrentPosition ""r2r4/p2p1p1p/b6R/n1p1kp2/2P2P2/3BP3/PP5P/4K2R b K f3 1 22""] [Timezone ""UTC""] [ECO ""E22""] [ECOUrl ""https://www.chess.com/openings/Nimzo-Indian-Defense-Spielmann-Variation""] [UTCDate ""2013.01.30""] [UTCTime ""16:35:14""] [WhiteElo ""1708""] [BlackElo ""1608""] [TimeControl ""1/259200""] [Termination ""-Amos- won by checkmate""] [StartTime ""16:35:14""] [EndDate ""2013.02.01""] [EndTime ""18:14:48""] [Link ""https://www.chess.com/game/daily/64629816""] 1. d4 Nf6 2. c4 e6 3. Nc3 Bb4 4. Qb3 Bxc3+ 5. Qxc3 O-O 6. Bg5 c5 7. dxc5 Nc6 8. Nf3 Qa5 9. Bxf6 gxf6 10. Qxa5 Nxa5 11. e3 Rd8 12. Rd1 Kg7 13. Be2 b6 14. Rd4 bxc5 15. Rg4+ Kh6 16. Bd3 f5 17. Rh4+ Kg6 18. g4 Ba6 19. gxf5+ exf5 20. Ne5+ Kf6 21. Rh6+ Kxe5 22. f4# 1-0"
-Amos-,koltcho69,https://api.chess.com/pub/player/-amos-,https://api.chess.com/pub/player/koltcho69,1726,1577,win,resigned,daily,1/172800,chess,true,8/5Q1k/4n1pp/8/7P/2N2b2/PP3P2/5K2 b - - 1 33,"[Event ""Rapid Rats - Board 5""] [Site ""Chess.com""] [Date ""2013.01.19""] [Round ""-""] [White ""-Amos-""] [Black ""koltcho69""] [Result ""1-0""] [Match ""https://www.chess.com/club/matches/219602""] [CurrentPosition ""8/5Q1k/4n1pp/8/7P/2N2b2/PP3P2/5K2 b - - 1 33""] [Timezone ""UTC""] [ECO ""C53""] [ECOUrl ""https://www.chess.com/openings/Giuoco-Piano-Game-Main-Line""] [UTCDate ""2013.01.19""] [UTCTime ""14:29:25""] [WhiteElo ""1726""] [BlackElo ""1577""] [TimeControl ""1/172800""] [Termination ""-Amos- won by resignation""] [StartTime ""14:29:25""] [EndDate ""2013.02.01""] [EndTime ""18:22:03""] [Link ""https://www.chess.com/game/daily/64070770""] 1. e4 e5 2. Nf3 Nc6 3. Bc4 Bc5 4. c3 a6 5. d4 exd4 6. cxd4 Be7 7. Qb3 Na5 8. Qc2 Nxc4 9. Qxc4 d6 10. Nc3 c6 11. O-O h6 12. Re1 Nf6 13. d5 c5 14. e5 dxe5 15. Nxe5 O-O 16. Ng6 Re8 17. Rxe7 Rxe7 18. Nxe7+ Qxe7 19. Bf4 b5 20. d6 Qd7 21. Qxc5 Bb7 22. Qc7 Qxc7 23. dxc7 Nd5 24. Rd1 Nxf4 25. Rd8+ Kh7 26. Rxa8 Bxa8 27. c8=Q Bxg2 28. Qxa6 Bf3 29. Qxb5 Nh3+ 30. Kf1 g6 31. Qd7 Ng5 32. h4 Ne6 33. Qxf7+ 1-0"
-Amos-,enhmandah,https://api.chess.com/pub/player/-amos-,https://api.chess.com/pub/player/enhmandah,1727,842,win,resigned,daily,1/172800,chess,true,rn1q1b1r/kb2p1pp/2p5/p1Q5/N1BP2n1/4PN2/1P3PPP/R1B1K2R b KQ - 5 15,"[Event ""CHESS BOARD CLASH - Round 1""] [Site ""Chess.com""] [Date ""2013.02.01""] [Round ""-""] [White ""-Amos-""] [Black ""enhmandah""] [Result ""1-0""] [Tournament ""https://www.chess.com/tournament/just-another-clash""] [CurrentPosition ""rn1q1b1r/kb2p1pp/2p5/p1Q5/N1BP2n1/4PN2/1P3PPP/R1B1K2R b KQ - 5 15""] [Timezone ""UTC""] [ECO ""D00""] [ECOUrl ""https://www.chess.com/openings/Queens-Pawn-Opening-1...d5-2.e3""] [UTCDate ""2013.02.01""] [UTCTime ""11:24:19""] [WhiteElo ""1727""] [BlackElo ""842""] [TimeControl ""1/172800""] [Termination ""-Amos- won by resignation""] [StartTime ""11:24:19""] [EndDate ""2013.02.02""] [EndTime ""17:58:11""] [Link ""https://www.chess.com/game/daily/64714474""] 1. d4 d5 2. e3 c6 3. c4 dxc4 4. Bxc4 b5 5. Bb3 a5 6. Qf3 Bb7 7. Bxf7+ Kd7 8. Qf5+ Kc7 9. Nf3 Nh6 10. Qe5+ Kb6 11. a4 bxa4 12. Nc3 Ng4 13. Nxa4+ Ka6 14. Bc4+ Ka7 15. Qc5+ 1-0"
enhmandah,-Amos-,https://api.chess.com/pub/player/enhmandah,https://api.chess.com/pub/player/-amos-,819,1727,checkmated,win,daily,1/172800,chess,true,r3kb1r/pp3ppp/3p1n2/2pKp3/P3P3/1P6/4qP1P/QNB5 w kq - 3 17,"[Event ""CHESS BOARD CLASH - Round 1""] [Site ""Chess.com""] [Date ""2013.02.01""] [Round ""-""] [White ""enhmandah

root
 |-- white_username: string (nullable = true)
 |-- black_username: string (nullable = true)
 |-- white_id: string (nullable = true)
 |-- black_id: string (nullable = true)
 |-- white_rating: integer (nullable = true)
 |-- black_rating: integer (nullable = true)
 |-- white_result: string (nullable = true)
 |-- black_result: string (nullable = true)
 |-- time_class: string (nullable = true)
 |-- time_control: string (nullable = true)
 |-- rules: string (nullable = true)
 |-- rated: boolean (nullable = true)
 |-- fen: string (nullable = true)
 |-- pgn: string (nullable = true)



## 2.5 Extrair dados da coluna PGN

In [0]:
# Função auxiliar para extrair tags do PGN (ex.: [ECO "B01"])
def pgn_tag(col_pgn, tag_name: str):
    return F.when(
        col_pgn.isNotNull(),
        F.regexp_extract(col_pgn, rf'\[{tag_name} "([^"]+)"\]', 1)
    ).otherwise(F.lit(None).cast("string"))


# Extrair tags principais do PGN
df_silver = (
    df_silver
    .withColumn("pgn_utc_date", pgn_tag(F.col("pgn"), "UTCDate"))
    .withColumn("pgn_date", pgn_tag(F.col("pgn"), "Date"))
    .withColumn("pgn_eco", pgn_tag(F.col("pgn"), "ECO"))
    .withColumn("pgn_eco_url", pgn_tag(F.col("pgn"), "ECOUrl"))
    .withColumn("pgn_termination", pgn_tag(F.col("pgn"), "Termination"))
    .withColumn("pgn_link", pgn_tag(F.col("pgn"), "Link"))
)

# Derivar game_date: prefer UTCDate, senão Date (formato comum: YYYY.MM.DD)
df_silver = (
    df_silver
    .withColumn(
        "game_date_raw",
        F.when(F.length(F.col("pgn_utc_date")) > 0, F.col("pgn_utc_date"))
         .when(F.length(F.col("pgn_date")) > 0, F.col("pgn_date"))
         .otherwise(F.lit(None).cast("string"))
    )
    .withColumn("game_date", F.to_date(F.regexp_replace(F.col("game_date_raw"), r"\.", "-"), "yyyy-MM-dd"))
)

print("Novas colunas:")
display(df_silver.select("pgn_utc_date","pgn_date","game_date_raw","game_date","pgn_eco","pgn_termination","pgn_link").limit(10))

Novas colunas:


pgn_utc_date,pgn_date,game_date_raw,game_date,pgn_eco,pgn_termination,pgn_link
2013.01.30,2013.01.30,2013.01.30,2013-01-30,E22,-Amos- won by checkmate,https://www.chess.com/game/daily/64629816
2013.01.19,2013.01.19,2013.01.19,2013-01-19,C53,-Amos- won by resignation,https://www.chess.com/game/daily/64070770
2013.02.01,2013.02.01,2013.02.01,2013-02-01,D00,-Amos- won by resignation,https://www.chess.com/game/daily/64714474
2013.02.01,2013.02.01,2013.02.01,2013-02-01,B20,-Amos- won by checkmate,https://www.chess.com/game/daily/64714344
2013.02.01,2013.02.01,2013.02.01,2013-02-01,A40,-Amos- won by resignation,https://www.chess.com/game/daily/64714476
2013.01.16,2013.01.16,2013.01.16,2013-01-16,C02,-Amos- won by resignation,https://www.chess.com/game/daily/63926510
2013.02.01,2013.02.01,2013.02.01,2013-02-01,,-Amos- won on time,https://www.chess.com/game/daily/64714452
2013.02.01,2013.02.01,2013.02.01,2013-02-01,A40,-Amos- won on time,https://www.chess.com/game/daily/64714486
2013.01.30,2013.01.30,2013.01.30,2013-01-30,A15,merlin68 won by resignation,https://www.chess.com/game/daily/64629842
2013.02.01,2013.02.01,2013.02.01,2013-02-01,B20,-Amos- won by resignation,https://www.chess.com/game/daily/64714362


## 2.6 Derivar colunas/atributos

In [0]:
draw_reasons = ["stalemate", "repetition", "agreed", "insufficient", "50move", "timevsinsufficient"]

df_silver = (
    df_silver
    .withColumn("rating_diff", F.col("white_rating") - F.col("black_rating"))
    .withColumn(
        "outcome",
        F.when(F.col("white_result") == F.lit("win"), F.lit("white_win"))
         .when(F.col("black_result") == F.lit("win"), F.lit("black_win"))
         .when(
             F.col("white_result").isin(draw_reasons) & F.col("black_result").isin(draw_reasons),
             F.lit("draw")
         )
         .otherwise(F.lit("unknown"))
    )
    .withColumn(
        "white_score",
        F.when(F.col("outcome") == "white_win", F.lit(1.0))
         .when(F.col("outcome") == "draw", F.lit(0.5))
         .when(F.col("outcome") == "black_win", F.lit(0.0))
         .otherwise(F.lit(None).cast("double"))
    )
    .withColumn(
        "black_score",
        F.when(F.col("outcome") == "black_win", F.lit(1.0))
         .when(F.col("outcome") == "draw", F.lit(0.5))
         .when(F.col("outcome") == "white_win", F.lit(0.0))
         .otherwise(F.lit(None).cast("double"))
    )
)

# termination_type: para vitórias, o motivo aparece no "result" do perdedor.
# Para draws/unknown, está em pgn_termination quando disponível. Fallback nos campos de resultado.
df_silver = (
    df_silver
    .withColumn(
        "termination_type",
        F.when(F.col("outcome") == "white_win", F.col("black_result"))
         .when(F.col("outcome") == "black_win", F.col("white_result"))
         .otherwise(F.col("pgn_termination"))
    )
    .withColumn(
        "termination_type",
        F.when(F.length(F.col("termination_type")) > 0, F.col("termination_type"))
         .otherwise(F.col("white_result"))
    )
)

display(df_silver.select("white_result","black_result","outcome","white_score","black_score","rating_diff","termination_type").limit(20))


white_result,black_result,outcome,white_score,black_score,rating_diff,termination_type
win,checkmated,white_win,1.0,0.0,100,checkmated
win,resigned,white_win,1.0,0.0,149,resigned
win,resigned,white_win,1.0,0.0,885,resigned
checkmated,win,black_win,0.0,1.0,-908,checkmated
win,resigned,white_win,1.0,0.0,613,resigned
win,resigned,white_win,1.0,0.0,179,resigned
timeout,win,black_win,0.0,1.0,-190,timeout
win,timeout,white_win,1.0,0.0,190,timeout
win,resigned,white_win,1.0,0.0,130,resigned
resigned,win,black_win,0.0,1.0,-628,resigned


## 2.7 Criar Game Key (para dedupe)

In [0]:
# Construir game_key com fallback (só usa colunas que existirem)
candidate_cols = []
for c in ["pgn_link", "game_url", "url", "link"]:   # o dataset pode variar
    if c in df_silver.columns:
        candidate_cols.append(F.col(c))

if candidate_cols:
    game_key = F.coalesce(*candidate_cols)
else:
    # fallback extremo: hash do PGN (se existir) ou concat de ids+time
    if "pgn" in df_silver.columns:
        game_key = F.sha2(F.col("pgn"), 256)
    else:
        game_key = F.sha2(F.concat_ws("||", F.col("white_id").cast("string"), F.col("black_id").cast("string"), F.col("time_control").cast("string")), 256)

df_silver = df_silver.withColumn("game_key", game_key)

n_before = df_silver.count()
df_silver_dedup = df_silver.dropDuplicates(["game_key"])
n_after = df_silver_dedup.count()

print("Linhas antes do dedup:", n_before)
print("Linhas depois do dedup:", n_after)
print("Dedup removeu:", n_before - n_after)

# Checks: domínios e nulos (principais)
display(
    df_silver_dedup.groupBy("outcome").count().orderBy(F.desc("count"))
)

display(
    df_silver_dedup.groupBy("time_class").count().orderBy(F.desc("count"))
)

# Null checks (campos principais)
qc_cols = ["white_rating", "black_rating", "white_result", "black_result", "time_class", "rated", "pgn_eco", "termination_type", "game_key"]
qc_exprs = []
for c in qc_cols:
    if c in df_silver_dedup.columns:
        qc_exprs.append(F.sum(F.when(F.col(c).isNull(), 1).otherwise(0)).alias(f"null_{c}"))

display(df_silver_dedup.agg(*qc_exprs))


Linhas antes do dedup: 66879
Linhas depois do dedup: 66875
Dedup removeu: 4


outcome,count
white_win,33349
black_win,31215
draw,2311


time_class,count
blitz,29468
bullet,22501
rapid,12933
daily,1973


null_white_rating,null_black_rating,null_white_result,null_black_result,null_time_class,null_rated,null_pgn_eco,null_termination_type,null_game_key
0,0,0,0,0,0,0,0,0


## 2.8 Salvar a Silver Table

In [0]:
silver_table = "workspace.default.silver_games"

(df_silver_dedup.write
 .format("delta")
 .mode("overwrite")
 .saveAsTable(silver_table)
)

print("Silver salvo em:", silver_table)


Silver salvo em: workspace.default.silver_games


## SQL Queries de teste

In [0]:
%sql
SELECT COUNT(*) AS n_rows FROM workspace.default.silver_games;


n_rows
66875


In [0]:
%sql
SELECT outcome, COUNT(*) AS n
FROM workspace.default.silver_games
GROUP BY outcome
ORDER BY n DESC;


outcome,n
white_win,33349
black_win,31215
draw,2311


In [0]:
%sql
SELECT *
FROM workspace.default.silver_games
LIMIT 10;


white_username,black_username,white_id,black_id,white_rating,black_rating,white_result,black_result,time_class,time_control,rules,rated,fen,pgn,pgn_utc_date,pgn_date,pgn_eco,pgn_eco_url,pgn_termination,pgn_link,game_date_raw,game_date,rating_diff,outcome,white_score,black_score,termination_type,game_key
Search-Goa,01KURA_50DACH1,https://api.chess.com/pub/player/search-goa,https://api.chess.com/pub/player/01kura_50dach1,1324,1257,win,resigned,bullet,60,chess,true,rn1qk2r/pbpp1ppp/1p2p3/8/1b1PN3/3B1N2/PPPB1PPP/R2QK2R b KQkq -,"[Event ""Live Chess""] [Site ""Chess.com""] [Date ""2018.08.05""] [Round ""-""] [White ""Search-Goa""] [Black ""01KURA_50DACH1""] [Result ""1-0""] [CurrentPosition ""rn1qk2r/pbpp1ppp/1p2p3/8/1b1PN3/3B1N2/PPPB1PPP/R2QK2R b KQkq -""] [Timezone ""UTC""] [ECO ""B00""] [ECOUrl ""https://www.chess.com/openings/Kings-Pawn-Opening-Owens-Defense""] [UTCDate ""2018.08.05""] [UTCTime ""05:54:59""] [WhiteElo ""1324""] [BlackElo ""1257""] [TimeControl ""60""] [Termination ""Search-Goa won by resignation""] [StartTime ""05:54:59""] [EndDate ""2018.08.05""] [EndTime ""05:55:20""] [Link ""https://www.chess.com/game/live/2985168387""] 1. e4 {[%clk 0:00:59.9]} 1... b6 {[%clk 0:00:59.6]} 2. Nc3 {[%clk 0:00:59.8]} 2... Bb7 {[%clk 0:00:59.5]} 3. Nf3 {[%clk 0:00:59.4]} 3... e6 {[%clk 0:00:58.6]} 4. d4 {[%clk 0:00:58.9]} 4... Bb4 {[%clk 0:00:57.9]} 5. Bd3 {[%clk 0:00:57.8]} 5... Nf6 {[%clk 0:00:55.6]} 6. Bd2 {[%clk 0:00:56.7]} 6... Nxe4 {[%clk 0:00:49.9]} 7. Nxe4 {[%clk 0:00:54.9]} 1-0",2018.08.05,2018.08.05,B00,https://www.chess.com/openings/Kings-Pawn-Opening-Owens-Defense,Search-Goa won by resignation,https://www.chess.com/game/live/2985168387,2018.08.05,2018-08-05,67,white_win,1.0,0.0,resigned,https://www.chess.com/game/live/2985168387
09Gladiator09,johnhodgmansucks,https://api.chess.com/pub/player/09gladiator09,https://api.chess.com/pub/player/johnhodgmansucks,1008,1024,win,timeout,bullet,60,chess,true,8/6p1/1R2k2p/8/8/2p4P/1r4PK/8 b - -,"[Event ""Live Chess""] [Site ""Chess.com""] [Date ""2018.11.17""] [Round ""-""] [White ""09Gladiator09""] [Black ""johnhodgmansucks""] [Result ""1-0""] [CurrentPosition ""8/6p1/1R2k2p/8/8/2p4P/1r4PK/8 b - -""] [Timezone ""UTC""] [ECO ""A00""] [ECOUrl ""https://www.chess.com/openings/Mieses-Opening-Reversed-Rat-Variation""] [UTCDate ""2018.11.17""] [UTCTime ""15:13:18""] [WhiteElo ""1008""] [BlackElo ""1024""] [TimeControl ""60""] [Termination ""09Gladiator09 won on time""] [StartTime ""15:13:18""] [EndDate ""2018.11.17""] [EndTime ""15:15:14""] [Link ""https://www.chess.com/game/live/3223394231""] 1. d3 {[%clk 0:00:59.9]} 1... e5 {[%clk 0:00:59.9]} 2. e3 {[%clk 0:00:59.5]} 2... d5 {[%clk 0:00:56.5]} 3. Bd2 {[%clk 0:00:59.1]} 3... Bc5 {[%clk 0:00:54.9]} 4. Be2 {[%clk 0:00:58.8]} 4... d4 {[%clk 0:00:54.2]} 5. exd4 {[%clk 0:00:57.5]} 5... Bxd4 {[%clk 0:00:53.5]} 6. c3 {[%clk 0:00:57.1]} 6... Bb6 {[%clk 0:00:49.4]} 7. Be3 {[%clk 0:00:56.6]} 7... Nc6 {[%clk 0:00:46.8]} 8. Bxb6 {[%clk 0:00:55.4]} 8... cxb6 {[%clk 0:00:46.5]} 9. Nf3 {[%clk 0:00:54.7]} 9... Nf6 {[%clk 0:00:44.9]} 10. O-O {[%clk 0:00:53.9]} 10... Bg4 {[%clk 0:00:44.5]} 11. Re1 {[%clk 0:00:53.4]} 11... Bxf3 {[%clk 0:00:43.4]} 12. Bxf3 {[%clk 0:00:52.4]} 12... O-O {[%clk 0:00:42.8]} 13. Bxc6 {[%clk 0:00:51.6]} 13... bxc6 {[%clk 0:00:41.4]} 14. Nd2 {[%clk 0:00:51.2]} 14... Qc7 {[%clk 0:00:36.4]} 15. Rc1 {[%clk 0:00:50.7]} 15... h6 {[%clk 0:00:35.6]} 16. f3 {[%clk 0:00:50]} 16... Nh7 {[%clk 0:00:34.7]} 17. Ne4 {[%clk 0:00:49.4]} 17... f5 {[%clk 0:00:32]} 18. Nd2 {[%clk 0:00:47.8]} 18... Ng5 {[%clk 0:00:30]} 19. d4 {[%clk 0:00:47.1]} 19... e4 {[%clk 0:00:28.4]} 20. fxe4 {[%clk 0:00:45.1]} 20... fxe4 {[%clk 0:00:27.4]} 21. Nxe4 {[%clk 0:00:44.4]} 21... Nxe4 {[%clk 0:00:24.3]} 22. Rxe4 {[%clk 0:00:43.6]} 22... c5 {[%clk 0:00:21.8]} 23. Qd3 {[%clk 0:00:42.4]} 23... cxd4 {[%clk 0:00:19.7]} 24. Qxd4 {[%clk 0:00:41.9]} 24... Qb7 {[%clk 0:00:18.5]} 25. Rf1 {[%clk 0:00:39.2]} 25... Rfd8 {[%clk 0:00:15.9]} 26. Qe3 {[%clk 0:00:37.5]} 26... b5 {[%